<font size=3 color='blue'>
    
  # <center>Clase 2, septiembre 29 del 2021</center> 

<font size=3 color='blue'>
    
  # <center>Ajuste de un conjunto de puntos con una función</center> 

<font size=2 color='blue'>
    
# Generación de las muestras

<font size=4 >
$$ $$
Se genera un conjunto de $m$ numeros aleatorios ($(x_1,y_1),(x_2,y_2),…,(x_m,y_m)$)

In [ ]:
# Importing the python libraries

import numpy as np
import random
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
%matplotlib inline

# To have a reproducible code we use a random seed 
np.random.seed(3)

In [ ]:
# Esta función genera un conjunto de datos que simulan 
# la medición de la distancia de un carrito en un riel de aire
# en la ausencia de una fuerza sobre el carrito.
# Se propone un error en la medición de la distancia

def generador_datos_simple(n_points, distance_0, measuring_time, speed, max_distance_error):
    
    """
      n_points: number of point that will be generated, integer
      distance_0 : initial distantce (at time zero) 
      measuring_time: the time inteval used for the measurement
      speed : carś speed
      max_distance_error: Maximum error measuring distance
      
    """
    
    # n_points es el número de puntos que serán generados
    
    x = np.random.random(n_points) * measuring_time
     
    # x es arreglo con n_points numeros aleatorios entre 0.0 y measuring_time
    
    error = np.random.randn(n_points) * max_distance_error 
    
    # error es un error generado aleatoriamente con un valor maximo max_distance_error

    y = distance_0 + speed*x + error 
        
    return x, y


In [ ]:
# Generacción de las muestras (xi,yi)
n_points = 1000
distance_0 = 100.0
measure_time = 100.0
speed = 20.0
max_distance_error = 100

x, y = generador_datos_simple(n_points, distance_0, measure_time, speed, max_distance_error)

print("x type", type(x), "x shape", x.shape)
print("y type", type(y), "y shape", y.shape)

x = np.squeeze(x)
y = np.squeeze(y)
print("x type", type(x), "x shape", x.shape)
print("y type", type(y), "y shape", y.shape)

In [ ]:
# Plotting y versus x

plt.figure(figsize=(13,8))
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
plt.rc('legend', fontsize=16)
plt.ylabel('Y(cm)', fontsize=16)
plt.xlabel('X(seg)', fontsize=16)

plt.scatter(x, y)
plt.show()

<font size=2 color='blue'>

# Buscando la correlación entre las muestras:



<font size=4>

Se tiene un conjunto de muestras (puntos) $(x_i, y_i)$, y se busca encontrar una función $F$ que describa una posible correlación entre ellos. $X$, con valores $x_i$ (en el presente caso el tiempo) es una variable independiente, mientras que $Y$, con los valores $y_i$ (la distancia en el presente caso) depende de $X$.

Para encontrar la correlación entre las muestras, proponemos un conjunto de funciones definidas mediante la siguiente relación lineal:

$$ F(X, W, b) = b + W X $$

en donde los parámetros $b$ y $W$ son variables que deben ser actualizadas hasta encontrar los valores que definan la función que mejor describa la correlación entre $X$ y $Y$. $$ $$
Vemos que esta relación funcional es derivable respecto a todas sus variables, $X, W, b$.
La letra $W$ se emplea como abreviación de la palabra en ingles "weight", porque se relaciona con la importancia que tiene la variable $X$ en el valor de la función $F$. La letra b es la abreviación de la palabra "bias" en ingles, $Y$ se refiere a la referencia respecto a cero de la función $F$.


<font size = 4>

Para encontrar la función que describe la correlación entre los puntos, es necesario generar una métrica para describir qué tanto se acerca cada una de las funciones específicas (con valores definidos de $W$ y de $b$) a esta descripción.

La métrica que se propone es la siguiente: $$ $$

Para cada muestra $(x_i, y_i)$ se evalua $F(x_i,W,b)$ y se compara con el correspondiente valor $y_i$, la diferencia entre estos valores se eleva al cuadrado. $$ $$
   $$ (F(x_i,w,b)-y_i)^{2}$$


Finalmente se calcula el promedio de este valor sobre todas las muestras, el cual definimos como error cuadrático medio (MSE, por sus siglas en ingles, Mean Squared Error). 
Si m es el número de muestras, el MSE queda como:

$$MSE = \dfrac {1}{m}∑_{i=1}^{m}(F(x_i,w,b)-y_i)^{2} $$

<font size=4, color='blue'>
    
En el siguiente código se implementa la generación del error cuadrático medio dada una función específica definida por los pesos iniciales $W$= weight_0, y $b$ = bias_0.

In [ ]:
#Initializing the variables of the function f

weight_0 = 10.0
bias_0 = 100.0

<font size=4 color="blue">

Se grafica la correspondiente función $F(X,W,b)$, junto con los puntos que representan a las muestras

In [ ]:
# Function for plotting (x,y) and (x,y_)

def plot_x_y_y__(x, y, y_, points=True):
    
    plt.figure(figsize=(13,8))
    plt.rc('xtick', labelsize=16)
    plt.rc('ytick', labelsize=16)
    plt.rc('legend', fontsize=16)
    plt.ylabel('Y(cm)', fontsize=16)
    plt.xlabel('X(seg)', fontsize=16)

    #Plotting function
    plt.plot(x, y_, color='green', lw=4, label='F(X, W, b)')
    plt.legend()

    #Plotting samples
    plt.scatter(x, y)

    plt.show()
    
    

<font size=4>

Se grafica la correspondiente función f para los valores $x_i$, así como las muestras $(x_i, y_i)$

In [ ]:
#The following arrays are generated for plotting the Function F(x, weight_0, bias_0)

y_ = weight_0*x + bias_0

# Using the function f, the residuos is calculated by comparing the calculated and measured values

residuo = np.mean((y-y_)**2)

print('residuo: {0:10.2f}'.format( residuo))

plot_x_y_y__(x, y, y_, points=True)

<font size=4>

De la gráficas se observa que la función está lejos de describir la correlación entre los puntos. 


<font size=4 color="blue">

Los valores del peso $W$ y el bias $b$ se actualizan iterativamente a prueba y error. Para ello

se hacen los cambios de acuerdo a la sugerencia de la observado en la gráfica. 

In [ ]:
def update_weights_biases(x, y, weight, bias, delta_weight, delta_bias):
    
    weight = weight + delta_weight
    
    bias = bias + delta_bias

    #The following date are for constructing the F(x,weight, bias)
    
    y_ = weight*x + bias
       
    residuo = np.mean((y - y_)**2)
    
    #print('residuo: {:10.2f}'.format(residuo))   
        
    return weight, bias, y_, residuo

In [ ]:
weight = weight_0
bias = bias_0
delta_w = 1.0
delta_b = -10.0

weight, bias, y_, residuo = update_weights_biases(x, y , weight, bias, delta_w, delta_b)
plot_x_y_y__(x, y, y_, points=True) 

<font size=4>

El cambio es muy pequeño, por ello el peso se actualiza con delta_weight = 2.0 y el bias con delta_bias= -100.0. 

In [ ]:
weight = weight
bias = bias
delta_w = 6.0
delta_b = -100.0

weight, bias, y_, residuo = update_weights_biases(x, y, weight, bias, delta_w, delta_b)
plot_x_y_y__(x, y, y_, points=True) 

<font size=4>

Aun se puede mejorar disminuyendo el peso (pendiente) y aumentando el bias (ordenada en el origen):

In [ ]:
weight = weight
bias = bias
delta_w = 1.5
delta_b = 50.0

weight, bias, y_, residuo = update_weights_biases(x,y, weight, bias, delta_w, delta_b)
plot_x_y_y__(x, y, y_, points=True) 

<font size=4>

Se puede mejorar el resultado. Para ello se observa el cambio en el residuo con los cambios en $W$ y en $b$.

<font size=4 color='blue'>
    
Busquemos encontrar la función que defina la correlación generando un conjunto de
    
funciones y calcular los correspondientes residuos.

<font size=4>

Para ello emplearemos la función **update_weight_bias()**, la cual actualiza los parametros $W$ y $b$.

In [ ]:
update_outputs = []

weight = weight_0
bias = -3.0
delta_w = 0.2 
delta_b = 0.0
iterations = 100

for i in range(iterations):
    
    weight, bias, y_, residuo = update_weights_biases(x, y, weight, bias, delta_w, delta_b)
    
    update_outputs.append([weight, bias, y_, residuo])
    
    if i % 10 == 0 :
            print('weight: {0:8.2f}    bias: {1:5.2f}   residuo: {2:10.2f}'.format(weight,bias, residuo))

In [ ]:
plt.figure(figsize=(13,8))
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
plt.rc('legend', fontsize=12)
plt.ylabel('Y', fontsize=16)
plt.xlabel('X', fontsize=16)

for i in range(0,50,3):

    plt.plot(x, update_outputs[i][0]*x + update_outputs[i][1], label='v' + str(i), lw=3)
    plt.legend()

plt.scatter(x, y)

plt.show()

<font size=4>

A continuación se grafica el residuo obtenido para cada función al variar el parámetro W.
 
 El valor óptimo del parámetro es aquel que minimiza al residuo. 

In [ ]:
plt.figure(figsize=(13,8))
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
plt.rc('legend', fontsize=16)
plt.ylabel('Residuo', fontsize=18)
plt.xlabel('W', fontsize=16)

weights = []

residuos = []

for update_output in update_outputs:
    
    weights.append(update_output[0])
    
    residuos.append(update_output[3])
        
plt.scatter(weights, residuos)


plt.show()

<font size = 4>

Como se observa en la grafica anterior, el mínimo del parámetro $W$ esta cerca de $W$= 22.

In [ ]:
index_min = np.argmin(residuos)   # returns the index for the minimum value in the list

print('El residuo mas pequeño tiene el valor: {0:10.2f}'.format(update_outputs[index_min][3]))

w_opt = update_outputs[index_min][0]
b_opt = update_outputs[index_min][1]

print('Los valores optimos de los parámetros son W = {0:5.2f} y b = {1:5.2f} ' \
      .format(w_opt, b_opt))

<font size = 4>

Se grafica la recta con los valores óptimos encontrados w_opt and b_opt :

In [ ]:
y_ = w_opt*x + b_opt

plot_x_y_y__(x, y, y_, points=True)

<font size=4 color='blue'>
Se genera la derivada parcial del residuo respecto a $W$ para cada valor de $W$ y se grafica.

In [ ]:
residuos_array = np.asarray(residuos)

In [ ]:
residuos_grad = np.gradient(residuos_array)
#residuos_grad

In [ ]:
plt.figure(figsize=(13,8))
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
plt.rc('legend', fontsize=16)
plt.ylabel('Derivada del Residuo', fontsize=16)
plt.xlabel('W', fontsize=16)
plt.grid(True)
plt.scatter(weights, residuos_grad)

plt.show()

<font size=4>

En este caso, como el residuo es una parábola, su derivada esta dada por la recta. 
    
$$ \dfrac{∂Residuo}{∂{W}} = -\dfrac {2}{m} ∑_{i=1}^{m}(y_i - b -Wx_i)x_i= -\dfrac {2}{m} ∑_{i=1}^{m}(y_i x_i- b x_i-Wx_i^{2})$$

$$ $$
En general para cualquier otro problema, el residuo tendrá una forma muy diferente al de una parábola

<font size=4 color='blue'>

Se generan un conjunto de funciones dejando $W$ fija y variando $b$.

In [ ]:
update_outputs = []

weight =  w_opt
bias = b_opt
delta_w = 0.0
delta_b = 1.0
iterations = 100

for i in range(iterations):
    
    weight, bias, y_, residuo = update_weights_biases(x, y, weight, bias, delta_w, delta_b)
    
    update_outputs.append([weight, bias, y_, residuo])
    
    if i % 10 == 0 :
        print('weight: {0:5.2f},   bias: {1:5.2f},   residuo: {2:8.2f}'.format(weight,bias,residuo))

In [ ]:
plt.figure(figsize=(13,8))
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
plt.rc('legend', fontsize=12)
plt.ylabel('Y', fontsize=16)
plt.xlabel('X', fontsize=16)

for i in range(0,50,3):

    plt.plot(x, w_opt*x + update_outputs[i][1], label='v' + str(i), lw=3)
    plt.legend()

plt.scatter(x, y)

plt.show()

<font size=4>

A continuación se grafica el residuo en función de $b$.
 
 El valor óptimo de $b$ es aquel que minimiza al residuo. 

In [ ]:
plt.figure(figsize=(13,8))
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
plt.rc('legend', fontsize=16)
plt.ylabel('Residuo', fontsize=16)
plt.xlabel('b', fontsize=16)

bias = []

residuos = []

for update_output in update_outputs:
    
    bias.append(update_output[1])
    
    residuos.append(update_output[3])
    
plt.scatter(bias, residuos)

plt.show()

<font size = 4>

Como se observa en la grafica el mínimo del residuo corresponde a un valor de $b$ cercano a 20.

In [ ]:
index_min = np.argmin(residuos)   # returns the index for the minimum value in the list
residuo_min = update_outputs[index_min][3]

print('El residuo mas pequeño tiene el valor: {0:10.2f}'.format(residuo_min))

w_opt = update_outputs[index_min][0]
b_opt = update_outputs[index_min][1]

print('Los valores optimos de los parámetros son W = {0:5.2f} y b = {1:5.2f} ' \
      .format(w_opt, b_opt))

<font size = 4>

Se grafica la recta empleando los valores óptimos $W$, w_opt, y $b$, b_opt.

In [ ]:
y_ = w_opt*x + b_opt
y_hand = y_

plot_x_y_y__(x, y, y_, points=True)

<font size=4 color='blue'>
Se genera la derivada parcial del residuo respecto a $W$ para cada valor de $W$ y se grafica.

In [ ]:
residuos_array = np.asarray(residuos)

In [ ]:
residuos_grad = np.gradient(residuos_array)
#residuos_grad

In [ ]:
plt.figure(figsize=(13,8))
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
plt.rc('legend', fontsize=16)
plt.ylabel('Derivada del Residuo', fontsize=18)
plt.xlabel('b', fontsize=18)

bias = []

residuos = []

for update_output in update_outputs:
    
    bias.append(update_output[1])
    
    residuos.append(update_output[3])

plt.grid(True)
plt.scatter(bias, residuos_grad)

plt.show()

<font size=4>

En este caso, como el residuo es una parábola, su derivada esta dada por la recta 
    
$$ \dfrac{∂Residuo}{∂{b}} = -\dfrac {2}{m} ∑_{i=1}^{m}(y_i - b -Wx_i)$$

$$ $$
En general para cualquier otro problema, el residuo tendrá una forma muy diferente al de una parábola

<font color = 'blue' size = 4>
    
## <center>Métodos alternativos para optimizar los parámetros $W$ y $b$</center>

<font color = 'blue' size = 4>

### Método simple de mínimos cuadrados, de acuerdo a Gauss

<font size=4>

El método de mínimos cuadrados nos permite encontrar una función que describe la correlación que tienen un conjunto m de puntos $(x_i, y_i)$, en donde $x_i$ son los valores que toma la variable $X$ y $y_i$ son los valores que toma la variable $Y$.


El objetivo del algoritmo es obtener la relación entre la variable independiente $X$, y la variable dependiente $Y$.

En el presente caso, se propone una función lineal para describir la correlación:

$$ F(X,W,b) = b + W X $$
 

Los parámetros variables $b$  y $W$ definen a la función $F$. 

Con el método simple de mínimos cuadrados se hallan los valores óptimos w_opt y b_opt de estas variables.

<font size = 4>

La diferencia entre el valor real $y_i$ de $Y$ y el valor estimado $F(x_i, W, b)$ se denomina el residuo ($r_i$):

$$ r_i = y_i - F(x_i, W, b) $$

El objetivo del método de mínimos cuadrados es minimizar la suma del cuadrado de estos  residuos, es decir, encontrar el mínimo de la función Residuo:

$$ Residuo = \dfrac {1}{m} ∑_{i=1}^{m}(y_i - b -Wx_i)^{2} $$

El mínimo de esta función, se encuentra en aquellos valores de $W$ ($W_{opt}$) y $b$ ($b_{opt}$) en donde la derivada de la función residuo es igual a zero. 
    
    
Para encontrar este mínimo, primero calculamos la derivada parcial de la función respecto a cada uno de los parámetros $W$ y $b$, después, cada derivada la igualamos a cero:

$$ \dfrac{∂Residuo}{∂{W}} =0⟶ W_{opt}=\dfrac{∑_{i=1}^{n}(x_i–\bar x)(y_i–\bar y)}{∑_{i=1}^{m}(x_i–\bar x)^2}$$


$$ \dfrac{∂Residuo}{∂{b}}=0⟶ b_{opt}=y – W_{opt} x $$

en donde $\bar x$ y $\bar y$ son los valores promedio de los valores de las variables $X$ y $Y$.



In [ ]:
def mean_square_error(x, y):
    
    # 1) Se obtiene el promedio de los valores x_i y y_i

    mean_x = np.mean(x)

    mean_y = np.mean(y)

    # 2) se calcula (x_i-x) y (y_i-y), con x e y los promedios de x_i e y_i, respectivamente.

    x_i = []

    for i in x:

        x_i.append(np.squeeze(i) - mean_x)

    y_i = []

    for i in y:

        y_i.append(np.squeeze(i) - mean_y)

    # 3) se calcula (x_i-x)*(x_i-x)

    x_i2 = np.power(x_i, 2)

    # 4) se hacen las sumas correspondientes

    xy_sum = 0

    for i in range(len(x_i)):

        xy_sum += x_i[i]*y_i[i]

    x2_sum = 0

    for i in range(len(x_i)):

        x2_sum += x_i2[i]

    # Se definen a w y b 

    w = xy_sum/x2_sum

    b = mean_y - w*mean_x
       
    return w, b



In [ ]:

w_opt_mse, b_opt_mse = mean_square_error(x, y)

print(" w_opt = {0:5.2f}, b_opt = {1:5.2f}".format(w_opt_mse,b_opt_mse))

In [ ]:
y_ = w_opt_mse*x + b_opt_mse

plot_x_y_y__(x, y, y_, points=True)

In [ ]:
MSE = np.mean((y-w_opt*x-b_opt)**2)
print('MSE: {0:10.2f}'.format(MSE))

In [ ]:
y_MSE = x*w_opt + b_opt

plt.figure(figsize=(13,8))
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
plt.rc('legend', fontsize=16)
plt.ylabel('Y', fontsize=16)
plt.xlabel('X', fontsize=16)


plt.plot(x, y_hand, color='magenta', label='hand_MSE-residuo', lw=6)

plt.plot(x, y_MSE, color='blue', label = 'Gauss_MSE-residuo', lw=3)

plt.legend()

plt.scatter(x, y)

plt.show()


<font size=4>

Como vimos anteriormente, la función $F(X,W,b)$ que describe la correlacion entre las variables $X$ y $Y$, se obtiene definiendo la métrica descrita por la función $Residuo$.
Cuando esta función Residuo tiene su mínimo (el error mínimo) su derivada es cero. Es por ello, que podriamos emplear el método de Newton-Raphson para encontrar este cero.

<font size=5 color='blue'>
    
Método de Newton-Raphson para encontrar el cero de una función. 
    
Este método emplea la función y su derivada.
    


<font size=4>

Otra alternativa para encontrar los valores de W y b para los cuales el residuo es mínimo es emplear un método iterativo desarrollado por Isaac Newton y Joseph Raphson en el siglo XVII para encontrar los ceros de una función. Hoy en día, este método es conocido como el método de Newton-Raphson.
   
Para emplear este método en nuestro caso se tendría que obtener la derivada de la función Residuo. Los valores de $W$ y $b$ se actualizarían empleando la siguiente relación:   
    
 $$W_{new} = W_{actual} - \dfrac {Residuo} {\dfrac {∂Residuo}{∂{W}}}$$
    
 $$b_{new} = b_{actual} - \dfrac {Residuo} {\dfrac {∂Residuo}{∂{b}}}$$

<font size=4>

El código que generaremos para emplear este método requiere un valor inicial de la variable que define la función, así como una epsilon para estimar si el error ya se puede considerar como cero. El método tambien toma en cuenta el número máximo de iteraciones que se deben realizar para encontrar este cero.

<font size=4>

Para introducir este método emplearemos la función $f(x)$ que depende de $x$, y de la cual queremos obtener sus ceros. La expresión correspondiente queda como:


 $$x_{new} = x_{actual} - \dfrac {f(x)} {\dfrac {df(x)}{d{x}}}$$
    

<img src = "NewtonIteration_Ani.gif">

(By Ralf Pfeifer - de:Image:NewtonIteration Ani.gif, CC BY-SA 3.0, https://commons.wikimedia.org/w/index.php?curid=2268473)

<font size=4>
A continuación definimos la función newton_raphson para implementar este método

In [ ]:
def newton_raphson(f, Df,x0,epsilon,max_iter):
    '''Approximate solution of f(x)=0 by Newton's method.

    Parameters
    ----------
    f : function
        
    Df : Derivative of f(x).
    
    x0 : Initial guess for finding the root of f(x).
    
    epsilon :Stopping criteria: the iteration ends when abs(f(x)) < epsilon.
    
    max_iter : Maximum number of iterations of Newton's method.

    Returns
    -------
    xn : number
        Implement Newton's method: compute the linear approximation
        of f(x) at xn and find x intercept by the formula
            x = xn - f(xn)/Df(xn)
        Continue until abs(f(xn)) < epsilon and return xn.
        If Df(xn) == 0, return None. If the number of iterations
        exceeds max_iter, then return None.
    '''
    
    aprox_root = [x0]
    
    xn = x0
      
    # xn es la aproximación de la raíz de f. Inicialmente xn =x0 con x0 la primera aproximación
    
    for n in range(0,max_iter):
        
        fxn = f(xn)
       
        #print("xn = ", xn, "aprox_root = ", aprox_root)
        
        if abs(fxn) < epsilon:
            
            print("x = ", xn, ", f(x) = ", fxn, ", df(x)/dx = ", Dfxn)
            print('The solution is found after',n,'iterations.')
                        
            return xn, aprox_root
        
        Dfxn = Df(xn)
        
        if Dfxn == 0:
            
            print('Zero derivative. No solution found.')
            
            return None
        
        print("x = ", xn, ", f(x) = ", fxn, ", df(x)/dx = ", Dfxn)
        
        xn = xn - fxn/Dfxn
        
        aprox_root.append(xn)
        
    print('Exceeded maximum iterations. No solution found.')
    
    return None

<font size=4>

Se prueba este método con una función simple

In [ ]:
#p = lambda x: x**3-27

def p(x):
    
    return x**3-10

In [ ]:
x_ = np.arange(-10, 10, 0.2)
print (x_.shape)
sigma_samples = 5
y_ = p(x_)
print(y_[:5])

plt.figure(figsize=(13,8))
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
plt.rc('legend', fontsize=16)
plt.ylabel('Y', fontsize=16)
plt.xlabel('X', fontsize=16)


plt.grid(True)

plt.plot(x_, y_, "o", ms=5, alpha= 0.5, color='r')

plt.show()

#print(y_)

<font size=4 color='blue'>
Calculemos la derivada de la función

In [ ]:
#Dp = lambda x: 3*x**2 

def Dp(x):
    
    return 3*x**2



<font size=4 color='blue'>
Para visualizar el método dibujemos la pendiente en cada punto en donde esta se calcula durante la búsqueda del cero.

In [ ]:
def tangent_line(f, Df, x_0, a, b):
        
    x = np.linspace(a,b)
    
    y = f(x) 
    
    y_0 = f(x_0)
    
    y_tan = Df(x_0) * (x - x_0) + y_0 
    
    plt.figure(figsize=(13,8))
    plt.rc('xtick', labelsize=16)
    plt.rc('ytick', labelsize=16)
    plt.rc('legend', fontsize=16)
    plt.ylabel('Y', fontsize=16)
    plt.xlabel('X', fontsize=16)



    #plt.plot(x,y,'r-')
    plt.plot(x_, y_, "o", ms=5, alpha= 0.5, color='r')
    
    plt.plot(x,y_tan,'b-')
    
    plt.xlabel('x') 
    
    plt.ylabel('y') 
    
    plt.grid(True)
    
    plt.title('Plot of the function and its tangent at x') 
    
    plt.show()  

In [ ]:

for i in newton_raphson(p,Dp,7.5,0.001,50)[1]:

    tangent_line(p, Dp, i, -10, 10)

<font size=4>

Para aplicar este método en nuestro caso, es necesario implementarlo para dos variables ($W$ y $b$). Sin embargo, en nuestro caso el gradiente del Residuo varía linealmente con cada una de estas variables. Su pendiente es entonces constante.

El método en general ha sido implementado para problemas con multivariables (número de variables mayor a 2)

[Newton-Raphson Multivariate](./Literatura/Newton-Raphson-multivariate.pdf)

<font size=4>

Otra alternativa para encontrar los valores de $W$ y $b$ para los cuales el residuo es mínimo, es emplear un método iterativo desarrollado por Cauchy en el siglo XIX. Hoy en día este método de optimización es conocido como el método de gradiente descendente.
    
[Cauchy, Gradiente descendente](./Literatura/Cauchy_gradient-descent.pdf)
    
En este caso durante la optimización, los valores de los parámetros $W$ y $b$ cambian su valor disminuyéndolo con el respectivo valor del gradiente del residuo multiplicado par el factor $\alpha$ (> 0).
    
 $$W_{new} = W_{actual} - \alpha \dfrac{∂Residuo}{∂{W}}$$
    
 $$b_{new} = b_{actual} - \alpha \dfrac{∂Residuo}{∂{b}}$$